In [3]:
import os 

In [4]:
%pwd

'/home/devil/Text-Summerization-/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/home/devil/Text-Summerization-'

In [31]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [32]:
from TextSummerizer.constants import * 
from TextSummerizer.utils.common import read_yaml, create_directories

In [33]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config

In [34]:

from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk,load_metric
import torch
import pandas as pd 
from tqdm import tqdm

In [35]:
class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_element, batch_size):
        """
        split the dataset into smaller batches that we can process simultaneously
        yields successive batch-sized chunks from list of elements
        """
        for i in range(0, len(list_of_element), batch_size):
            yield list_of_element[i:i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16, device='cuda' if torch.cuda.is_available() else "cpu",
                                    column_text="Text",
                                    column_summary="Summary"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
                zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs['attention_mask'].to(device),
                                       length_penalty=0.8, num_beams=8, max_length=128)
            """
            parameter for the length penalty ensures that the model doesn't sequence that are too long
            """

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                                  clean_up_tokenization_spaces=True)
                                 for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score

    def evaluate(self):
        device = 'cuda' if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        dataset = load_from_disk(self.config.data_path)
        
        rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(dataset['test'][:10], rouge_metric, model_pegasus,
                                                 tokenizer, batch_size=2, column_text='Text', column_summary='Summary')

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        
        df.to_csv(self.config.metric_file_name, index=False)


In [36]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2023-12-19 17:57:58,562: INFO: common: YAML file 'config/config.yaml' loaded successfully]
[2023-12-19 17:57:58,567: INFO: common: YAML file 'params.yaml' loaded successfully]
[2023-12-19 17:57:58,568: INFO: common: Created directory at: artifacts]
[2023-12-19 17:57:58,570: INFO: common: Created directory at: artifacts/model_evaluation]


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'artifacts/model_trainer/tokenizer'. Use `repo_type` argument if needed.